<a href="https://colab.research.google.com/github/rlarkdtks312/nutrition_4_team/blob/master/%5B%EA%B8%B0%EB%8A%A5_%EA%B5%AC%ED%98%84%5D_%EC%84%AD%EC%B7%A8_%EC%98%81%EC%96%91%EC%86%8C%EC%99%80_%EA%B8%B0%EC%A4%80_%EC%98%81%EC%96%91%EC%86%8C_%EB%B9%84%EA%B5%90_%EA%B8%B0%EB%8A%A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### SQLite

In [ ]:
!apt install sqlite3

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Suggested packages:
  sqlite3-doc
The following NEW packages will be installed:
  sqlite3
0 upgraded, 1 newly installed, 0 to remove and 39 not upgraded.
Need to get 752 kB of archives.
After this operation, 2,482 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 sqlite3 amd64 3.22.0-1ubuntu0.4 [752 kB]
Fetched 752 kB in 0s (5,943 kB/s)
Selecting previously unselected package sqlite3.
(Reading database ... 155455 files and directories currently installed.)
Preparing to unpack .../sqlite3_3.22.0-1ubuntu0.4_amd64.deb ...
Unpacking sqlite3 (3.22.0-1ubuntu0.4) ...
Setting up sqlite3 (3.22.0-1ubuntu0.4) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd

### data1 영양권장량

In [ ]:
data1 = pd.read_excel('/content/drive/MyDrive/영양4조/남진영/[최종]영양권장량.xlsx')
data1

/usr/local/lib/python3.7/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,성별,연령,영양성분,권장(g/일),상한(g/일)
0,남자,19-29,에너지(kcal),2600,-
1,남자,30-39,에너지(kcal),2500,-
2,남자,40-49,에너지(kcal),2500,-
3,남자,50-64,에너지(kcal),2200,-
4,여자,19-29,에너지(kcal),2000,-
...,...,...,...,...,...
91,남자,50-64,아연(mg/일),10,35
92,여자,19-29,아연(mg/일),8,35
93,여자,30-39,아연(mg/일),8,35
94,여자,40-49,아연(mg/일),8,35


### 음식영양소 db 조회 food_nutrition= 기준영양소

In [ ]:
import sqlite3

In [ ]:
conn = sqlite3.connect('/content/drive/MyDrive/영양4조/[db]food_nutrition.db')
conn

In [ ]:
with conn:
    cur = conn.cursor()
    sql = "select * from food_nutrition where 음식명=?"
    cur.execute(sql, ('쌀밥',))
    rows = cur.fetchall()
    
    for row in rows:
        print(row)

('쌀밥', 210.0, 334.8, 73.71, 0.0, 0.45, 5.76, 3.6, 126.0, 59.4, 146.7, 0.0, 0.36, 1.35, 0.0, 0.0)


In [ ]:
food_list = ['쌀밥', '근대된장국', '떡갈비']

In [ ]:
def food_nutri(food_list) : 
    
    conn = sqlite3.connect('/content/drive/MyDrive/영양4조/[db]food_nutrition.db')

    with conn:
        cur = conn.cursor()
        sql = "select * from food_nutrition where 음식명=?"
        for i in food_list:
            food = (i,)
            cur.execute(sql, food)
            rows = cur.fetchall()
            cols = [column[0] for column in cur.description]
    data_df = pd.DataFrame.from_records(data=rows, columns=cols)

    return data_df

In [ ]:
food_nutri(food_list)

,음식명,중량(g),에너지(kcal),탄수화물(g),당류(g),지방(g),단백질(g),칼슘(mg),인(mg),나트륨(mg),칼륨(mg),마그네슘(mg),철(mg),아연(mg),콜레스테롤(mg),트랜스지방(g)
0,떡갈비,250.0,762.99,26.61,12.49,51.58,43.09,26.18,411.76,838.84,915.68,0.0,3.26,8.66,157.5,0.0


In [ ]:
conn = sqlite3.connect('/content/drive/MyDrive/영양4조/[db]food_nutrition.db')
sql = ("select * from food_nutrition")
ms = pd.read_sql(sql, conn)
ms.dropna(inplace=True)
ms.replace('-','0',inplace=True)
ms

,음식명,중량(g),에너지(kcal),탄수화물(g),당류(g),지방(g),단백질(g),칼슘(mg),인(mg),나트륨(mg),칼륨(mg),마그네슘(mg),철(mg),아연(mg),콜레스테롤(mg),트랜스지방(g)
0,쌀밥,210.0,334.80,73.71,0.00,0.45,5.76,3.60,126.00,59.40,146.70,0.00,0.36,1.35,0.00,0.0
1,기타잡곡밥,200.0,302.36,65.52,0.00,0.75,6.71,12.06,117.84,3.39,166.72,6.90,1.74,1.83,0.00,0.0
2,콩밥,200.0,322.90,65.85,0.00,1.67,8.43,24.70,126.90,4.10,258.40,22.50,1.74,1.61,0.00,0.0
3,보리밥,200.0,316.10,70.57,0.00,0.14,5.55,6.70,68.80,4.50,100.60,0.00,1.26,1.41,0.00,0.0
4,돌솥밥,350.0,528.86,101.85,0.00,8.35,10.19,61.46,193.72,618.32,624.32,57.79,3.32,2.58,2.37,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,다식,30.0,105.17,20.81,6.46,1.73,3.55,1.43,100.07,3.85,206.71,0.00,0.52,0.04,0.00,0.0
396,약과,30.0,113.85,22.18,2.15,1.24,2.57,4.45,28.23,16.46,59.42,0.00,0.57,0.14,0.00,0.0
397,유과,30.0,129.06,24.14,1.12,3.50,0.36,1.21,3.60,7.06,0.03,0.00,0.24,0.20,0.00,0.0
398,산자,30.0,121.70,24.70,10.63,1.20,0.91,2.00,20.36,8.01,60.93,0.00,0.41,0.27,0.00,0.0


### id_record db 조회 id_record=섭취영양소

In [ ]:
import sqlite3

In [ ]:
conn = sqlite3.connect('/content/drive/MyDrive/영양4조/남궁정후/id_record1.db')
conn

In [ ]:
with conn:
    cur = conn.cursor()
    sql = "select * from id_record where id=?"
    cur.execute(sql, ('kangsan',))
    rows = cur.fetchall()
    
    for row in rows:
        print(row)

('kangsan', '2022-04-14', '남자', '19-29', 200.0, 650.0, 70.0, 3.0, 19.2, 5.2, 2.6, 96.0, 4.1, 142.0, 0.0, 0.26, 2.0, 0.0, 0.0)
('kangsan', '2022-04-16', '남자', '19-29', 210.0, 720.0, 80.0, 4.7, 16.3, 5.9, 4.6, 136.0, 3.0, 143.0, 0.0, 0.3, 1.6, 0.0, 0.0)
('kangsan', '2022-04-17', '남자', '19-29', 220.0, 750.0, 74.0, 3.3, 20.0, 6.1, 2.8, 96.0, 4.3, 151.0, 0.0, 0.3, 2.1, 0.0, 0.0)


In [ ]:
# 전체 데이터 조회
with conn:
    cur = conn.cursor()
    sql = "select * from id_record"
    cur.execute(sql)
    rows = cur.fetchall()
    
    for row in rows:
        print(row)

('kangsan', '2022-04-14', '남자', '19-29', 200.0, 650.0, 70.0, 3.0, 19.2, 5.2, 2.6, 96.0, 4.1, 142.0, 0.0, 0.26, 2.0, 0.0, 0.0)
('junghu', '2022-04-15', '여자', '30-39', 210.0, 750.0, 90.0, 4.0, 19.29, 7.2, 3.6, 126.0, 4.1, 166.72, 0.0, 0.26, 1.83, 0.0, 0.0)
('kangsan', '2022-04-16', '남자', '19-29', 210.0, 720.0, 80.0, 4.7, 16.3, 5.9, 4.6, 136.0, 3.0, 143.0, 0.0, 0.3, 1.6, 0.0, 0.0)
('jinyoung', '2022-04-16', '여자', '30-39', 180.0, 590.0, 62.0, 3.0, 10.0, 7.0, 8.0, 117.0, 3.0, 152.0, 0.0, 0.3, 1.6, 0.0, 0.0)
('kangsan', '2022-04-17', '남자', '19-29', 220.0, 750.0, 74.0, 3.3, 20.0, 6.1, 2.8, 96.0, 4.3, 151.0, 0.0, 0.3, 2.1, 0.0, 0.0)


###셀 병합

In [ ]:
data1,conn

(    성별     연령       영양성분 권장(g/일) 상한(g/일)
 0   남자  19-29  에너지(kcal)    2600       -
 1   남자  30-39  에너지(kcal)    2500       -
 2   남자  40-49  에너지(kcal)    2500       -
 3   남자  50-64  에너지(kcal)    2200       -
 4   여자  19-29  에너지(kcal)    2000       -
 ..  ..    ...        ...     ...     ...
 91  남자  50-64   아연(mg/일)      10      35
 92  여자  19-29   아연(mg/일)       8      35
 93  여자  30-39   아연(mg/일)       8      35
 94  여자  40-49   아연(mg/일)       8      35
 95  여자  50-64   아연(mg/일)       8      35
 
 [96 rows x 5 columns], <sqlite3.Connection at 0x7f2e4b58b3b0>)

# 데이터 =====
- 영양정보 DB 가져오기

In [ ]:
DATA = pd.read_excel('/content/drive/MyDrive/영양4조/dataset/nutritionDB/음식분류 AI 데이터 영양DB.xlsx', index_col='음 식 명')
DATA.head()

,중량(g),에너지(kcal),탄수화물(g),당류(g),지방(g),단백질(g),칼슘(mg),인(mg),나트륨(mg),칼륨(mg),마그네슘(mg),철(mg),아연(mg),콜레스테롤(mg),트랜스지방(g)
음 식 명,,,,,,,,,,,,,,,
쌀밥,210.0,334.800,73.7100,0.0,0.4500,5.7600,3.6,126.0,59.400,146.7,-,0.36,1.35,0.000,0.0
기타잡곡밥,200.0,302.362,65.5211,0.0,0.7462,6.7092,12.06,117.836,3.392,166.718,6.9,1.7447,1.8264,0.000,0.0
콩밥,200.0,322.900,65.8460,0.0,1.6660,8.4280,24.7,126.9,4.100,258.4,22.5,1.74,1.61,0.000,0.0
보리밥,200.0,316.100,70.5700,0.0,0.1400,5.5500,6.7,68.8,4.500,100.6,0.0,1.26,1.41,0.000,0.0
돌솥밥,350.0,528.855,101.8500,0.0,8.3470,10.1870,61.455,193.72,618.315,624.32,57.785,3.321,2.582,2.365,0.0


# 영양성분정보 추출 함수 

In [ ]:
# 음식명별 영양성분 정보
Nutrition = {'중량': ['중량(g)'],
             '성분': ['에너지(kcal)', '콜레스테롤(mg)', '트랜스지방(g)', '탄수화물(g)', '당류(g)', '지방(g)',
                    '단백질(g)', '칼슘(mg)', '인(mg)', '나트륨(mg)', '칼륨(mg)', '마그네슘(mg)', '철(mg)', '아연(mg)']}

In [ ]:
names = ['곤드레밥', '라면']

In [ ]:
# predict된 음식명에 따라 영양성분 출력 함수 정의
def nutrition_info(names):
    # 예측된 음식명 담기
    food_name = []
    # 없는 음식명 담기
    food_name_x = []
    # 예측된 음식의 영양성분 담기
    food_info = {}
    # 예측된 음식명들중에 하나씩 반복해서 이름은 음식명에 담고, 영양정보는 영양성분에 담기
    for name in names:
        if name in DATA.index:
            food_name.append(name)
            food_info[name] = dict(DATA.loc[name][Nutrition['성분']])
        # 음식명이 없으면 없는 음식명에 담고 없는 정보를 출력할 준비 함
        else:
            food_name_x.append(name)
            a = '없는 정보 :', food_name_x
    # 결과1: 없는음식명이 없다면 예측된 음식명 리스트와 그에대한 영양성분 정보 출력
    if food_name_x == []:
        return food_name, food_info
    # 결과2: 음식명이 없으면 없는음식명 정보를 출력
    elif food_name == []:
        return  print(a)
    # 결과3: 음식명이 있고, 없는것도 있으면 없는 음식명정보도 출력하고 예측된 음식명 리스트와 영양성분정보 출력
    else:
        print(a)
        return food_name, food_info

In [ ]:
nutrition_info(names)

NameError: ignored

In [ ]:
# 음식별 영양성분 더하기
def nutri_add(names):
    # 전체음식의 영양성분을 한번에 담을 변수 생성 Nutri
    Nutri = {'에너지(kcal)': 0, '콜레스테롤(mg)': 0, '트랜스지방(g)': 0, '탄수화물(g)': 0, '당류(g)': 0, '지방(g)': 0,
    '단백질(g)': 0, '칼슘(mg)': 0, '인(mg)': 0, '나트륨(mg)': 0, '칼륨(mg)': 0, '마그네슘(mg)': 0, '철(mg)': 0, '아연(mg)': 0}
    # 추출된 전체 음식의 성분별로 영양성분 더하여 dict에 저장
    for name in names[0]:
        for k in Nutrition['성분']:
            Nutri[k] += names[1][name][k]
    return Nutri

In [ ]:
result = nutri_add(nutrition_info(names))
result

NameError: ignored

# 가공정보 추출 함수 

In [ ]:
DATA2 = pd.read_excel('/content/drive/MyDrive/영양4조/남진영/[최종]영양권장량.xlsx')
DATA2

/usr/local/lib/python3.7/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,성별,연령,영양성분,권장(g/일),상한(g/일)
0,남자,19-29,에너지(kcal),2600.0,-
1,남자,30-39,에너지(kcal),2500.0,-
2,남자,40-49,에너지(kcal),2500.0,-
3,남자,50-64,에너지(kcal),2200.0,-
4,여자,19-29,에너지(kcal),2000.0,-
...,...,...,...,...,...
91,남자,50-64,아연(mg),10.0,35
92,여자,19-29,아연(mg),8.0,35
93,여자,30-39,아연(mg),8.0,35
94,여자,40-49,아연(mg),8.0,35


In [ ]:
DATA2[(DATA2['성별'] =='남자') & (DATA2['연령']=='19-29') & (DATA2['영양성분']=='탄수화물(g/일)')]

,성별,연령,영양성분,권장(g/일),상한(g/일)


In [ ]:
a = DATA2[(DATA2['성별'] == '남자') & (DATA2['연령']=='19-29')]
a[['영양성분','권장(g/일)']]

,영양성분,권장(g/일)
0,에너지(kcal),2600.000000
8,탄수화물(g),130.000000
16,당류(g),63.414634
24,지방(g),55.026455
32,단백질(g),65.000000
40,칼슘(mg),800.000000
48,인(mg),700.000000
56,나트륨(mg),1500.000000
64,칼륨(mg),3500.000000
72,마그네슘(mg),360.000000


In [ ]:
gender = '남자'
ages = 28
augments = '탄수화물(g)'

In [ ]:
result['탄수화물(g)']

191.22000000000003

In [ ]:
type(a[a['영양성분']=='에너지(kcal)']['권장(g/일)'].values[0])

numpy.float64

In [ ]:
Nutrition['성분']

['에너지(kcal)',
 '콜레스테롤(mg)',
 '트랜스지방(g)',
 '탄수화물(g)',
 '당류(g)',
 '지방(g)',
 '단백질(g)',
 '칼슘(mg)',
 '인(mg)',
 '나트륨(mg)',
 '칼륨(mg)',
 '마그네슘(mg)',
 '철(mg)',
 '아연(mg)']

In [ ]:
def mod(gender, age, names):
    df = DATA2[(DATA2['성별'] == gender) & (DATA2['연령']== age)]
    result = nutri_add(nutrition_info(names))
    for aug in df['영양성분']:
        if int(result[aug]) >= int(df[df['영양성분']==aug]['권장(g/일)']):
            c = int(result[aug])
            f = int(df[df['영양성분']==aug]['권장(g/일)'])
            g = aug.split('(')[1].split(')')[0]
            print(aug,'섭취량', round(c-f),g ,'초과!경고~')
        else:
            print(aug,'은(는) 안전~')
    return

In [ ]:
mod('남자', '19-29', names)

에너지(kcal) 은(는) 안전~
탄수화물(g) 섭취량 61 g 초과!경고~
당류(g) 은(는) 안전~
지방(g) 은(는) 안전~
단백질(g) 은(는) 안전~
칼슘(mg) 은(는) 안전~
인(mg) 은(는) 안전~
나트륨(mg) 섭취량 284 mg 초과!경고~
칼륨(mg) 은(는) 안전~
마그네슘(mg) 은(는) 안전~
철(mg) 은(는) 안전~
아연(mg) 은(는) 안전~


In [ ]:
if gender == '남자':
  if ages < 19:
    print('마니먹어..')
  elif ages < 30:
    a = DATA2[(DATA2['성별'] == '남자') & (DATA2['연령']=='19-29')]
    for aug in Nutrition['성분']:
      print(result[aug])
      print(a[a['영양성분']==[aug]]['권장(g/일)'].values[0]gm
      if int(result[aug]) >= a[a['영양성분']==[aug]]['권장(g/일)'].values[0]:
        print('경고')
      else:
        print('dd')

SyntaxError: ignored

In [ ]:
if gender == '남자':
    if ages < 19:
        print('마니먹어..')
    elif ages < 30:
        a = DATA2[(DATA2['성별'] == '남자') & (DATA2['연령']=='19-29')]
        for aug in Nutrition['성분']:
            if int(result[aug]) >= a[a['영양성분']==aug]['권장(g/일)'].values[0]:
                print('경고')
            else:
                print('dd')
    elif ages < 40:
        DATA2[(DATA2['성별'] == '남자') & (DATA2['연령']=='30-49')]
        print('남자 30대')
    elif ages < 50:
        DATA2[(DATA2['성별'] == '남자') & (DATA2['연령']=='30-49')]
        print('남자 40대')
    elif ages < 60:
        DATA2[(DATA2['성별'] == '남자') & (DATA2['연령']=='50-64')]
        print('남자 50대')
    elif ages < 65:
        DATA2[(DATA2['성별'] == '남자') & (DATA2['연령']=='50-64')]
        print('남자 60~64')
    else:
        print('해당 정보가 없습니다')
elif gender == '여자':
    if ages < 19:
        print('마니먹어..')
    elif ages < 30:
        DATA2[(DATA2['성별'] == '여자') & (DATA2['연령']=='19-29')]
        print('여자 20대')
    elif ages < 40:
        DATA2[(DATA2['성별'] == '여자') & (DATA2['연령']=='30-49')]
        print('여자 30대')
    elif ages < 50:
        DATA2[(DATA2['성별'] == '여자') & (DATA2['연령']=='30-49')]
        print('여자 40대')
    elif ages < 60:
        DATA2[(DATA2['성별'] == '여자') & (DATA2['연령']=='50-64')]
        print('여자 50대')
    elif ages < 65:
        DATA2[(DATA2['성별'] == '여자') & (DATA2['연령']=='50-64')]
        print('여자 60~64')
    else:
        print('해당 정보가 없습니다')

dd


IndexError: ignored

In [ ]:
# 입력받은 개인정보 기준 영양성분 제한 해주는 부분 함수
def warning(gender, ages, augments):
    a

# FASTAPI 설정 =====
- install fastapi, uvicorn 등
- import fastapi, ngrok 등

In [ ]:
!pip install fastapi nest-asyncio pyngrok uvicorn aiofiles python-multipart

     |████████████████████████████████| 54 kB 1.7 MB/s 
     |████████████████████████████████| 745 kB 10.0 MB/s 
     |████████████████████████████████| 53 kB 1.9 MB/s 
     |████████████████████████████████| 58 kB 5.1 MB/s 
     |████████████████████████████████| 10.9 MB 45.3 MB/s 
     |████████████████████████████████| 79 kB 6.9 MB/s 
     |████████████████████████████████| 58 kB 4.9 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=8161319f588dff48bebb856fab3783b86763ff8f206910e74c9fea620e3e5eb6
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
  Created wheel for python-multipart: filename=python_multipart-0.0.5-py3-none-any.whl size=31678 sha256=919d99f5d7da1cfc97257ac7d165953ea50f5acbf8510ccca95d4d735dd9941c
  Stored in directory: /root/.cache/pip/wheels/2c/41/7c/bfd1c180534ffdcc0972f78c5758f89881602175d48a8bcd2c
Successfully built pyngrok python-multipart


# HTML파일작성
- API 화면 출력용 html 작성

In [ ]:
%%writefile templates/main.html
<!DOCTYPE html>
<html>
	<head>
        <link rel="preconnect" href="https://fonts.googleapis.com" />
		<link rel="preconnect" href="https://fonts.gstatic.com" crossorigin />
		<link
			href="https://fonts.googleapis.com/css2?family=Gowun+Dodum&display=swap"
			rel="stylesheet"
		/>
        
        <!-- Required meta tags -->
		<meta charset="utf-8" />
		<meta name="viewport" content="width=device-width, initial-scale=1" />

        <!-- Bootstrap CSS -->
		<link
        href="https://cdn.jsdelivr.net/npm/bootstrap@5.1.3/dist/css/bootstrap.min.css"
        rel="stylesheet"
        integrity="sha384-1BmE4kWBq78iYhFldvKuhfTAU6auU8tT94WrHftjDbrCEXSU1oBoqyl2QvZ6jIW3"
        crossorigin="anonymous"
        />

        <title>Nutrition_4</title>
        <link rel="stylesheet" href="/static/style.css" />
    </head>
	<body>
        <form method="post" action="/print" enctype="multipart/form-data">
            <section class="section">
                <h2 class="title">개인맞춤 식단 필터링 서비스</h2>
                <h5 class="subtitle">정보를 입력해주세요</h5>
            <center>
            <div class= 'pt-1 Select Gender'>
                <select name = 'gender'>
                <option type='text', value='male'>남자</option>
                <option type='text', value='female'>여자</option>
                </select>
            </div>
            </center>
            <center>
            <div class= 'pt-1 Input Ages'>
                <input type='number' min='19' max="64" name="ages">
            </div>
            </center>
            <center>
            <div class= 'pt-1 Select Illness'>
                <select name = 'illness'>
                <option type='text', value='A'>비만</option>
                <option type='text', value='B'>당뇨</option>
                <option type='text', value='C'>고혈압</option>
                <option type='text', value='D'>고지혈증</option>
                <option type='text', value='E'>신부전증</option>
                </select>
            </div>
            </center>
            <center>
                <div class='pt-2 mb-2 submitbutton'>
                  <input type="submit", Value=작동>
                </div>
            </center>
            <div class="container file-upload">
                <div class="image-upload-wrap" style="width: 65%; margin: 0 auto;">
                    <input
                        class="file-upload-input"
                        type="file"
                        name='img'
                        onchange="readURL(this);"
                        accept="image/*"
                    />
                    <div class="drag-text">
                        <img src="img/upload.png" class="mt-5 pt-2 upload" style="width: 16%;" />
                        <h3 class="mb-5 pd-5 pt-4 upload-text">음식사진을 올려주세요</h3>
                    </div>
                </div>
                <div class="file-upload-content">
                    <img class="file-upload-image" id="food-image" src="#" alt="your image" />
                    <!-- <p class="result-message"></p> -->
                    <div id="label-container"></div>
                    <div class="image-title-wrap">
                        <button type="button" onclick="removeUpload()" class="remove-image">
                            Remove
                            <span class="image-title">Uploaded Image</span>
                        </button>
                    </div>
                </div>
            </div>
        </form>
        # <script>
		# 	function readURL(input) {
		# 		if (input.files && input.files[0]) {
		# 			var reader = new FileReader();
		# 			reader.onload = function (e) {
		# 				$('.image-upload-wrap').hide();
		# 				$('.file-upload-image').attr('src', e.target.result);
		# 				$('.file-upload-content').show();
		# 				$('.image-title').html(input.files[0].name);
		# 			};
		# 			reader.readAsDataURL(input.files[0]);
		# 			init().then(() => {
		# 				console.log('hello');
		# 				predict();
		# 			});
		# 		} else {
		# 			removeUpload();
		# 		}
		# 	}

		# 	function removeUpload() {
		# 		$('.file-upload-input').replaceWith($('.file-upload-input').clone());
		# 		$('.file-upload-content').hide();
		# 		$('.image-upload-wrap').show();
		# 	}
		# 	$('.image-upload-wrap').bind('dragover', function () {
		# 		$('.image-upload-wrap').addClass('image-dropping');
		# 	});
		# 	$('.image-upload-wrap').bind('dragleave', function () {
		# 		$('.image-upload-wrap').removeClass('image-dropping');
		# 	});
		# </script>    
    </body>
	
</html>

Overwriting templates/main.html


In [ ]:
!pip install fastapi nest-asyncio pyngrok uvicorn aiofiles python-multipart

     |████████████████████████████████| 54 kB 1.9 MB/s 
     |████████████████████████████████| 745 kB 17.9 MB/s 
     |████████████████████████████████| 53 kB 1.9 MB/s 
     |████████████████████████████████| 58 kB 5.7 MB/s 
     |████████████████████████████████| 10.9 MB 53.6 MB/s 
     |████████████████████████████████| 79 kB 8.0 MB/s 
     |████████████████████████████████| 58 kB 5.1 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=6ab549863d35ddeb81bc186e34fe8ca499f8a4a197599c2752169228b58df8fe
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
  Created wheel for python-multipart: filename=python_multipart-0.0.5-py3-none-any.whl size=31678 sha256=2a4a15d0337131ab21b39138f6fa05ac4af33f891b0d865df0b54a18c3c7c2bd
  Stored in directory: /root/.cache/pip/wheels/2c/41/7c/bfd1c180534ffdcc0972f78c5758f89881602175d48a8bcd2c
Successfully built pyngrok python-multipart


# css 파일 작성

In [ ]:
%%writefile static/style.css
body {
	font-family: 'Ubuntu', sans-serif;
	background-color: #fff;
}
.file-upload {
	/* width: 914px; */
	border-radius: 10px;
	border: solid 1.5px #f6f7fa;
	background-color: #f6f7fa;
	padding: 13% 5%;
	margin: 0 auto
	/* width: 600px; */
	
}
.file-upload-btn {
	width: 100%;
	margin: 0;
	color: #fff;
	background: #a61fb2;
	border: none;
	padding: 10px;
	border-radius: 4px;
	border-bottom: 4px solid #a61fb2;
	transition: all 0.2s ease;
	outline: none;
	text-transform: uppercase;
	font-weight: 700;
}
.file-upload-btn:hover {
	background: #1a1ca0;
	color: #fff;
	transition: all 0.2s ease;
	cursor: pointer;
}
.file-upload-btn:active {
	border: 0;
	transition: all 0.2s ease;
}
.file-upload-content {
	display: none;
	text-align: center;
}
.file-upload-input {
	position: absolute;
	margin: 0;
	padding: 0;
	width: 100%;
	height: 100%;
	outline: none;
	opacity: 0;
	cursor: pointer;
}
.image-upload-wrap {
	border: 4px dashed #0026ff;
	position: relative;
}
.image-upload-wrap:hover {
	background-color: #f9bbff;
	border: 4px dashed #fff;
}
.image-dropping {
	background-color: #ff0000;
	border: 4px dashed #fff;
}
.image-title-wrap {
	padding: 0 15px 15px;
	color: #222;
}
.drag-text {
	text-align: center;
}
.drag-text h3 {
	width: 50%;
	font-weight: 100;
	text-transform: uppercase;
	color: #15824b;
	font-family: 'Ubuntu', sans-serif;
	font-size: 1.5rem;
	line-height: 1.53;
	text-align: center;
	color: #35465d;
	margin: 0 auto;
}
.file-upload-image {
	max-height: 200px;
	max-width: 200px;
	margin: auto;
	padding: 20px;
}
.remove-image {
	width: 200px;
	margin: 0;
	color: #fff;
	background: #cd4535;
	border: none;
	padding: 10px;
	border-radius: 4px;
	border-bottom: 4px solid #b02818;
	transition: all 0.2s ease;
	outline: none;
	text-transform: uppercase;
	font-weight: 700;
}
.remove-image:hover {
	background: #c13b2a;
	color: #fff;
	transition: all 0.2s ease;
	cursor: pointer;
}
.remove-image:active {
	border: 0;
	transition: all 0.2s ease;
}

.nav-distance {
	padding: 2.1% 7.7%;
}

.navbar-light .navbar-brand {
	font-family: 'Ubuntu', sans-serif;
	line-height: 1.68;
	text-align: left;
	color: #35465d;
}

.navbar-light .navbar-toggler {
	border-color: #fff;
}

.section {
	margin-top: 100px;
}

.title {
	font-family: 'Ubuntu', sans-serif;
	line-height: 1.67;
	text-align: center;
	color: #35465d;
}

.subtitle {
	font-family: 'Ubuntu', sans-serif;
	line-height: 1.53;
	text-align: center;
	color: #35465d;
}

/* On screens that are 600px or less, set the background color to olive */
@media screen and (max-width: 600px) {
  html {
    font-size: 10px;
  }
	.navbar-brand {
		font-size: 2rem;
	}
}

.github-link {
	font-family: 'Ubuntu', sans-serif;
	font-weight: bolder;
	font-size: 1.0rem;
	line-height: 1.71;
	text-align: center;
	color: #000000;
	text-decoration: underline;
}

.github-link: hover {
	font-family: 'Ubuntu', sans-serif;
	font-weight: bolder;
	font-size: 1.0rem;
	line-height: 1.71;
	text-align: center;
	color: #2fb004;
	text-decoration: underline;
}

.INFOODD-icon {
	width: 10%
}




.image-upload-wrap {
	object-fit: contain;
	border-radius: 10px;
	border: dashed 1.5px
		#35465d;
	background-color:
		#ffffff;
}

.image-upload-wrap hover {
	object-fit: contain;
	border-radius: 10px;
	border: dashed 1.5px
		#35465d;
	background-color:
		#35465d;
}

.upload-text {
	width: 50%;
	font-family: 'Ubuntu', sans-serif;
	font-size: 1.6 rem;
	line-height: 1.53;
	text-align: center;
	color: #35465d;
	margin: 0 auto;
}

Writing static/style.css


# print.html

In [ ]:
%%writefile templates/print.html
<!DOCTYPE html>
<html>
	<head>
        <link rel="preconnect" href="https://fonts.googleapis.com" />
		<link rel="preconnect" href="https://fonts.gstatic.com" crossorigin />
		<link
			href="https://fonts.googleapis.com/css2?family=Ubuntu&display=swap"
			rel="stylesheet"
		/>
        
        <!-- Required meta tags -->
		<meta charset="utf-8" />
		<meta name="viewport" content="width=device-width, initial-scale=1" />

        <!-- Bootstrap CSS -->
		<link
        href="https://cdn.jsdelivr.net/npm/bootstrap@5.1.3/dist/css/bootstrap.min.css"
        rel="stylesheet"
        integrity="sha384-1BmE4kWBq78iYhFldvKuhfTAU6auU8tT94WrHftjDbrCEXSU1oBoqyl2QvZ6jIW3"
        crossorigin="anonymous"
        />

        <title>Nutrition_4</title>
        <link rel="stylesheet" href="/static/style.css" />
    </head>
	<body>
        <form method="post" action="/print" enctype="multipart/form-data">
            <section class="section">
                <h2 class="title">개인 맞춤형 식단 필터링 서비스</h2>
            <center>
            
            </center>
    </body>
	
</html>

Writing templates/print.html


# FastAPI 실행
- 연결 가능

In [ ]:
from typing import Optional
from pydantic import BaseModel

In [ ]:
# FastAPI - 변수설정 및 수정 필요
from fastapi.staticfiles import StaticFiles

class Select_Gender(BaseModel):
    img: UploadFile = File(...)
    gender: str = Form(...)
    ages: int = Form(...)
    illness: Optional[str] = Form(...)

app=FastAPI()
templates = Jinja2Templates(directory="templates")
app.mount('/static', StaticFiles(directory='static'), name='static')
app.mount('/img', StaticFiles(directory='img'), name='img')

@app.get('/', response_class=HTMLResponse)
async def read_page(request: Request):
  return templates.TemplateResponse("main.html", {"request": request, 'item':Item })

@app.post('/print', response_class=HTMLResponse)
async def get_page(request: Request):
  return templates.TemplateResponse("print.html", {"request": request})

In [ ]:
#ngrok authtoken
!ngrok authtoken 26P542M5d2txQcLcbZiAZnJa2Uv_ZWHtNDRbdXZKKnUuFqRQ

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
ngrok_tunnel = ngrok.connect(8000)
print ('Public URL:', ngrok_tunnel.public_url) 
nest_asyncio.apply()
uvicorn.run(app, host='0.0.0.0', port=8000)

Public URL: http://f6c0-34-73-227-199.ngrok.io


INFO:     Started server process [63]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     49.142.75.97:0 - "GET / HTTP/1.1" 200 OK
INFO:     49.142.75.97:0 - "GET /style.css HTTP/1.1" 200 OK
INFO:     49.142.75.97:0 - "GET / HTTP/1.1" 200 OK
INFO:     49.142.75.97:0 - "GET /upload.png HTTP/1.1" 404 Not Found
INFO:     49.142.75.97:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [63]


In [ ]:
@app.get('/', response_class=HTMLResponse)
async def read_page(request: Request):
  return templates.TemplateResponse("index.html", {"request": request})


In [ ]:
app=FastAPI()
templates = Jinja2Templates(directory="templates")
app.mount('/static', StaticFiles(directory='static'), name='static')
app.mount('/img', StaticFiles(directory='img'), name='img')


In [ ]:
!pip install fastapi nest-asyncio pyngrok uvicorn aiofiles python-multipart

# 모듈 임포트
from fastapi import FastAPI, Request, Form, File, UploadFile
from fastapi.responses import HTMLResponse
from fastapi.staticfiles import StaticFiles
from fastapi.templating import Jinja2Templates
import nest_asyncio
from pyngrok import ngrok

import uvicorn
